In [1]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer
model_name = 'cointegrated/rubert-tiny2-sentence-compression'
model = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


def compress(text, threshold=0.5, keep_ratio=None):
    """ Compress a sentence by removing the least important words.
    Parameters:
        threshold: cutoff for predicted probabilities of word removal
        keep_ratio: proportion of words to preserve
    By default, threshold of 0.5 is used.
    """
    with torch.inference_mode():
        tok = tokenizer(text, return_tensors='pt').to(model.device)
        proba = torch.softmax(model(**tok).logits, -1).cpu().numpy()[0, :, 1]
    if keep_ratio is not None:
        threshold = sorted(proba)[int(len(proba) * keep_ratio)]
    kept_toks = []
    keep = False
    prev_word_id = None
    for word_id, score, token in zip(tok.word_ids(), proba, tok.input_ids[0]):
        if word_id is None:
            keep = True
        elif word_id != prev_word_id:
            keep = score < threshold
        if keep:
            kept_toks.append(token)
        prev_word_id = word_id
    return tokenizer.decode(kept_toks, skip_special_tokens=True)


Downloading:   0%|          | 0.00/805 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/111M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/379 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.30M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [2]:
text = 'Документом глава государства закрепил статус многодетной для семей, в которых воспитываются трое и более детей. В частности, президентский указ гарантирует: пособия и выплаты в связи с рождением и воспитанием детей, меры поддержки в трудовой сфере, досрочную пенсию матери, профессиональное обучение родителей и возможность получить дополнительное образование. Руководству регионов рекомендовано дополнительно обеспечивать детей из многодетных семей бесплатным питанием в учебных заведениях и школьной формой, бесплатным проездом в транспорте, а детей до шести лет — лекарствами. Теперь регионы будут вправе расширить категорию многодетной семьи с учетом демографических, культурных и национальных особенностей субъекта.'
print(text)
print(compress(text), end='\n\n\n')
print(compress(text, threshold=0.3), end='\n\n\n')
print(compress(text, threshold=0.1), end='\n\n\n')


print("keep_ratio=0.5", compress(text, keep_ratio=0.5))


Документом глава государства закрепил статус многодетной для семей, в которых воспитываются трое и более детей. В частности, президентский указ гарантирует: пособия и выплаты в связи с рождением и воспитанием детей, меры поддержки в трудовой сфере, досрочную пенсию матери, профессиональное обучение родителей и возможность получить дополнительное образование. Руководству регионов рекомендовано дополнительно обеспечивать детей из многодетных семей бесплатным питанием в учебных заведениях и школьной формой, бесплатным проездом в транспорте, а детей до шести лет — лекарствами. Теперь регионы будут вправе расширить категорию многодетной семьи с учетом демографических, культурных и национальных особенностей субъекта.
бесплатным.


.


.


keep_ratio=0.5) Документом глава государства закрепил статус многодетной для семей, в воспитываются гарантирует : пособия и выплаты в с рождением и воспитанием детей, меры поддержки в трудовой сфере, досрочную пенсию матери, и семей бесплатным питанием в за

In [3]:
from transformers import MBartTokenizer, MBartForConditionalGeneration

model_name = "IlyaGusev/mbart_ru_sum_gazeta"
tokenizer = MBartTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)




Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/406 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/287 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.23G [00:00<?, ?B/s]

In [4]:
article_text = "Высота башни составляет 324 метра (1063 фута), примерно такая же высота, как у 81-этажного здания, и самое высокое сооружение в Париже. Его основание квадратно, размером 125 метров (410 футов) с любой стороны. Во время строительства Эйфелева башня превзошла монумент Вашингтона, став самым высоким искусственным сооружением в мире, и этот титул она удерживала в течение 41 года до завершения строительство здания Крайслер в Нью-Йорке в 1930 году. Это первое сооружение которое достигло высоты 300 метров. Из-за добавления вещательной антенны на вершине башни в 1957 году она сейчас выше здания Крайслер на 5,2 метра (17 футов). За исключением передатчиков, Эйфелева башня является второй самой высокой отдельно стоящей структурой во Франции после виадука Мийо."

input_ids = tokenizer(
    [article_text],
    max_length=1000,
    padding="max_length",
    truncation=True,
    return_tensors="pt",
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    no_repeat_ngram_size=2
)[0]

summary = tokenizer.decode(output_ids, skip_special_tokens=True)
print(summary)

Эйфелева башня стала самым высоким искусственным сооружением в мире, достигнув высоты 324 метра (1063 фута), примерно такая же высота, как у 81-этажного здания, и самое высокое в Париже.
